<a href="https://colab.research.google.com/github/newsun87/google-colab-line_bot_learning/blob/main/linebot_flexmessag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Flex Message 設計
它是一種彈性的訊息格式，自由的透過 JSON 設計出各種訊息類型，而不受限於標準的訊息內容 (例如文字、貼圖、template)

In [ ]:
!pip install flask_ngrok pyngrok
!pip install Flask==2.0.3
!pip install line-bot-sdk
!ngrok authtoken 2Sh3hFOuE9JYeMNTsS6EzBxHPIL_6xdGTubFt7b9kSeUDFVYH

#### Flex Message 結構
整個訊息結構是由 container 到 block 到 component 組合成的巢狀結構。

#### 複製 Flex Message Simulator 範例的 JSON code 發送 Flex Message


In [3]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *
import json
import os, time, requests

def get_ngrokurl():
    os.system("ps aux | grep ngrok.exe | awk '{print $2}' | xargs kill -9")
    os.system("nohup ngrok http 5000&")
    time.sleep(3)
    public_url = json.loads(requests.get('http://localhost:4040/api/tunnels').text)['tunnels'][0]['public_url']
    return public_url

ngrok_url = get_ngrokurl()
print(ngrok_url)
# 你的 LINE Channel access token'
access_token = 'hea6hI/LTc4FCcgIlR51IbcAjPWqy9QtsG5XX9Ce+4GwdfJH1EKl5/YOT3Hw4+aZOGWZSKcKPmEnUqzE4tOoBjPdvyVJJjIHwm9RN9/cWbwT7Je2GW7uNtaTSKysDrcW8F/BSEFJpywQo5Z9DXry6gdB04t89/1O/w1cDnyilFU='
# 你的 LINE Channel access token
channel_secret = '77ff5430fad4531107ca11f1c495e7dd'
# Channel Access Token
line_bot_api = LineBotApi(access_token)
# Channel Secret
line_handler = WebhookHandler(channel_secret)
# 設定 webhook url
line_bot_api.set_webhook_endpoint(ngrok_url+'/callback')

app = Flask(__name__)
# 監聽所有來自 LINE Bot Channel 的 Post Request
@app.route("/callback", methods=['POST'])
def linebot():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # 取得傳來的訊息文字
    body = request.get_data(as_text=True)
    # 轉換訊息內容成 JSON 物件
    json_data = json.loads(body)
    print(f"訊息內容 {json_data}")
    # 驗證訊息正確性
    try:
        line_handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

flex_message_json = {
      "type": "bubble",
      "hero": { # 設定主要圖片區塊
        "type": "image", # 建立圖片配置元件
        "url": "https://scdn.line-apps.com/n/channel_devcenter/img/fx/01_1_cafe.png",
        "size": "full",
        "aspectRatio": "20:13",
        "aspectMode": "cover",
        "action": {
          "type": "uri",
          "uri": "http://linecorp.com/"
        }
      },
      "body": { # 設定主要內容區塊
        "type": "box", # 建立矩形區域元件
        "layout": "vertical",
        "contents": [ # 建立區域內容
          {
            "type": "text",
            "text": "Brown Cafe",
            "weight": "bold",
            "size": "xl"
          },
          {
            "type": "box", # 建立矩形區域元件
            "layout": "baseline",
            "margin": "md",
            "contents": [
              {
                "type": "icon",
                "size": "sm",
                "url": "https://scdn.line-apps.com/n/channel_devcenter/img/fx/review_gold_star_28.png"
              },
              {
                "type": "icon",
                "size": "sm",
                "url": "https://scdn.line-apps.com/n/channel_devcenter/img/fx/review_gold_star_28.png"
              },
              {
                "type": "icon",
                "size": "sm",
                "url": "https://scdn.line-apps.com/n/channel_devcenter/img/fx/review_gold_star_28.png"
              },
              {
                "type": "icon",
                "size": "sm",
                "url": "https://scdn.line-apps.com/n/channel_devcenter/img/fx/review_gold_star_28.png"
              },
              {
                "type": "text",
                "text": "4.0",
                "size": "sm",
                "color": "#999999",
                "margin": "md",
                "flex": 0
              }
            ]
          },
          {
            "type": "box", # 建立矩形區域元件
            "layout": "vertical",
            "margin": "lg",
            "spacing": "sm",
            "contents": [ # 建立區域內容
              {
                "type": "box",
                "layout": "baseline",
                "spacing": "sm",
                "contents": [
                  {
                    "type": "text",
                    "text": "Place",
                    "color": "#aaaaaa",
                    "size": "sm",
                    "flex": 1
                  },
                  {
                    "type": "text",
                    "text": "Miraina Tower, 4-1-6 Shinjuku, Tokyo",
                    "wrap": True,
                    "color": "#666666",
                    "size": "sm",
                    "flex": 5
                  }
                ]
              },
              {
                "type": "box", # 建立矩形區域元件
                "layout": "baseline",
                "spacing": "sm",
                "contents": [ # 建立區域內容
                  {
                    "type": "text",
                    "text": "Time",
                    "color": "#aaaaaa",
                    "size": "sm",
                    "flex": 1
                  },
                  {
                    "type": "text",
                    "text": "10:00 - 23:00",
                    "wrap": True,
                    "color": "#666666",
                    "size": "sm",
                    "flex": 5
                  }
                ]
              }
            ]
          }
        ]
      },
      "footer": { # 設定底部內容區塊
        "type": "box", # 建立矩形區域元件
        "layout": "vertical",
        "spacing": "sm",
        "contents": [ # 建立區域內容
          {
            "type": "button",
            "style": "link",
            "height": "sm",
            "action": {
              "type": "uri",
              "label": "CALL",
              "uri": "https://linecorp.com"
            }
          },
          {
            "type": "button",
            "style": "link",
            "height": "sm",
            "action": {
              "type": "uri",
              "label": "WEBSITE",
              "uri": "https://linecorp.com"
            }
          },
          {
            "type": "box",
            "layout": "vertical",
            "contents": [],
            "margin": "sm"
          }
        ],
        "flex": 0
      }
    }

# 處理訊息
@line_handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
   if event.message.text == 'flexmessage':
       flex_message = FlexSendMessage(
         alt_text = "彈性配置範例",
         contents = flex_message_json
    )
   line_bot_api.reply_message(event.reply_token, flex_message)

if __name__ == "__main__":
  app.run()

https://908a-34-68-152-36.ngrok-free.app


<ipython-input-3-a3b722d64ee8>:23: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
<ipython-input-3-a3b722d64ee8>:25: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_handler = WebhookHandler(channel_secret)
<ipython-input-3-a3b722d64ee8>:27: LineBotSdkDeprecatedIn30: Call to deprecated method set_webhook_endpoint. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).set_webhook_endpoint(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '471450170618282312', 'text': 'flexmessage'}, 'webhookEventId': '01H9G9ABA4Q7S0A1PZH2JXKMGT', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693837503667, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': 'd8449ae7c35047248c26370ba8dc46de', 'mode': 'active'}]}


<ipython-input-3-a3b722d64ee8>:203: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, flex_message)
INFO:werkzeug:127.0.0.1 - - [04/Sep/2023 14:25:04] "POST /callback HTTP/1.1" 200 -


1. 修改 FLEX MESSAGE SIMULATOR 範本
2. 複製 JSON code 建立 Flex Message 版面
3. 回覆 Flex Message 訊息給用戶

In [9]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *
import json
import os, time, requests

def get_ngrokurl():
    os.system("ps aux | grep ngrok.exe | awk '{print $2}' | xargs kill -9")
    os.system("nohup ngrok http 5000&")
    time.sleep(3)
    public_url = json.loads(requests.get('http://localhost:4040/api/tunnels').text)['tunnels'][0]['public_url']
    return public_url

ngrok_url = get_ngrokurl()
print(ngrok_url)
# 你的 LINE Channel access token'
access_token = 'hea6hI/LTc4FCcgIlR51IbcAjPWqy9QtsG5XX9Ce+4GwdfJH1EKl5/YOT3Hw4+aZOGWZSKcKPmEnUqzE4tOoBjPdvyVJJjIHwm9RN9/cWbwT7Je2GW7uNtaTSKysDrcW8F/BSEFJpywQo5Z9DXry6gdB04t89/1O/w1cDnyilFU='
# 你的 LINE Channel access token
channel_secret = '77ff5430fad4531107ca11f1c495e7dd'
# Channel Access Token
line_bot_api = LineBotApi(access_token)
# Channel Secret
line_handler = WebhookHandler(channel_secret)
# 設定 webhook url
line_bot_api.set_webhook_endpoint(ngrok_url+'/callback')

app = Flask(__name__)
# 監聽所有來自 LINE Bot Channel 的 Post Request
@app.route("/callback", methods=['POST'])
def linebot():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # 取得傳來的訊息文字
    body = request.get_data(as_text=True)
    # 轉換訊息內容成 JSON 物件
    json_data = json.loads(body)
    print(f"訊息內容 {json_data}")
    # 驗證訊息正確性
    try:
        line_handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 複製 Flex Message Simulator 產生 JSON 資料
flex_message_json = {
  "type": "bubble",
  "hero": {
    "type": "image",
    "url": "https://i.imgur.com/YWjnVr2.jpg",
    "size": "full",
    "aspectRatio": "20:13",
    "aspectMode": "cover",
    "action": {
      "type": "uri",
      "uri": "http://linecorp.com/"
    }
  },
  "body": {
    "type": "box",
    "layout": "vertical",
    "contents": [
      {
        "type": "text",
        "text": "等一個人的咖啡",
        "weight": "bold",
        "size": "xl"
      },
      {
        "type": "box",
        "layout": "baseline",
        "margin": "md",
        "contents": [
          {
            "type": "icon",
            "size": "sm",
            "url": "https://scdn.line-apps.com/n/channel_devcenter/img/fx/review_gold_star_28.png"
          },
          {
            "type": "icon",
            "size": "sm",
            "url": "https://scdn.line-apps.com/n/channel_devcenter/img/fx/review_gold_star_28.png"
          },
          {
            "type": "icon",
            "size": "sm",
            "url": "https://scdn.line-apps.com/n/channel_devcenter/img/fx/review_gold_star_28.png"
          },
          {
            "type": "icon",
            "size": "sm",
            "url": "https://scdn.line-apps.com/n/channel_devcenter/img/fx/review_gold_star_28.png"
          },
          {
            "type": "icon",
            "size": "sm",
            "url": "https://scdn.line-apps.com/n/channel_devcenter/img/fx/review_gold_star_28.png"
          },
          {
            "type": "text",
            "text": "5.0",
            "size": "sm",
            "color": "#999999",
            "margin": "md",
            "flex": 0
          }
        ]
      },
      {
        "type": "box",
        "layout": "vertical",
        "margin": "lg",
        "spacing": "sm",
        "contents": [
          {
            "type": "box",
            "layout": "baseline",
            "spacing": "sm",
            "contents": [
              {
                "type": "text",
                "text": "地點",
                "color": "#6265a0",
                "size": "md",
                "flex": 1
              },
              {
                "type": "text",
                "text": "新北市淡水區中山路90巷20號",
                "color": "#6265a0",
                "size": "md",
                "flex": 5
              }
            ]
          },
          {
            "type": "box",
            "layout": "baseline",
            "spacing": "sm",
            "contents": [
              {
                "type": "text",
                "text": "時間",
                "color": "#6265a0",
                "size": "md",
                "flex": 1
              },
              {
                "type": "text",
                "text": "10:00 - 23:00",
                "color": "#6265a0",
                "size": "md",
                "flex": 5
              }
            ]
          }
        ]
      }
    ],
    "backgroundColor": "#34ffc2"
  },
  "footer": {
    "type": "box",
    "layout": "vertical",
    "spacing": "sm",
    "contents": [
      {
        "type": "button",
        "style": "link",
        "height": "sm",
        "action": {
          "type": "uri",
          "label": "CALL",
          "uri": "https://maps.app.goo.gl/KtRaVBkeDXWE9TXt9"
        }
      },
      {
        "type": "button",
        "style": "link",
        "height": "sm",
        "action": {
          "type": "uri",
          "label": "WEBSITE",
          "uri": "https://maps.app.goo.gl/KtRaVBkeDXWE9TXt9"
        }
      },
      {
        "type": "box",
        "layout": "vertical",
        "contents": [],
        "margin": "sm"
      }
    ],
    "flex": 0,
    "backgroundColor": "#8ed3f2"
  }
}

# 文字訊息事件處理程序
@line_handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
   if event.message.text == 'flexmessage':
       flex_message = FlexSendMessage(
         alt_text = "彈性配置範例",
         contents = flex_message_json
    )
   line_bot_api.reply_message(event.reply_token, flex_message)

if __name__ == "__main__":
  app.run()

https://5824-34-68-152-36.ngrok-free.app


<ipython-input-9-ee5635def5c9>:23: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
<ipython-input-9-ee5635def5c9>:25: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_handler = WebhookHandler(channel_secret)
<ipython-input-9-ee5635def5c9>:27: LineBotSdkDeprecatedIn30: Call to deprecated method set_webhook_endpoint. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).set_webhook_endpoint(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '471455391654347106', 'text': 'flexmessage'}, 'webhookEventId': '01H9GC9A6Q6RRBH82DT31S79XH', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693840615494, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': '1e02f534669447f5991ceebd8d6e277e', 'mode': 'active'}]}


<ipython-input-9-ee5635def5c9>:208: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, flex_message)
INFO:werkzeug:127.0.0.1 - - [04/Sep/2023 15:16:56] "POST /callback HTTP/1.1" 200 -
